In [2]:
import sympy as sp
import sympy.physics.mechanics as me

# creating constants
m,g,IUxx,IUyy,IUzz,IUxy,IUyz,IUzx,IMxx,IMyy,IMzz,IMxy, IMyz, IMzx,ILxx,ILyy,ILzz,ILxy, ILyz, ILzx,c,k = sp.symbols('m g IUxx IUyy IUzz IUxy IUyz IUzx IMxx IMyy IMzz IMxy IMyz IMzx ILxx ILyy ILzz ILxy ILyz ILzx c k')
x_cont1,y_cont1,z_cont1,x_cont2,y_cont2,z_cont2, k_contact, mx, my, mz, ax, ay, az, eps  = sp.symbols('x_cont1 y_cont1 z_cont1 x_cont2 y_cont2 z_cont2 k_contact mx my mz ax ay az eps')

# creating generalized coordinates, generalized forces and their derivatives
q1, q2, q3, q4, q5, q6, q7, q8, q9 = me.dynamicsymbols('q1 q2 q3 q4 q5 q6 q7 q8 q9')
q1d, q2d, q3d, q4d, q5d, q6d, q7d, q8d, q9d = me.dynamicsymbols('q1 q2 q3 q4 q5 q6 q7 q8 q9', 1)
u1, u2, u3, u4, u5, u6, u7, u8, u9 = me.dynamicsymbols('u1 u2 u3 u4 u5 u6 u7 u8 u9')
u1d, u2d, u3d, u4d, u5d, u6d, u7d, u8d, u9d = me.dynamicsymbols('u1 u2 u3 u4 u5 u6 u7 u8 u9', 1)
t = sp.Symbol('t')

# joint position from child and parent CoM (rigid1C = )
rigid0P = sp.symbols('rigid0Px:z')
rigid1P = sp.symbols('rigid1Px:z')
rigid1C = sp.symbols('rigid1Cx:z')
rigid2P = sp.symbols('rigid2Px:z')
rigid2C = sp.symbols('rigid2Cx:z')
rigid3C = sp.symbols('rigid3Cx:z')

# creating bodies
mc1 = me.Point('mc1')
mc2 = me.Point('mc2')
mc3 = me.Point('mc3')
frame1 = me.ReferenceFrame('frame1')
frame2 = me.ReferenceFrame('frame2')
frame3 = me.ReferenceFrame('frame3')
m1, m2, m3 = sp.symbols('m1 m2 m3')
thorax = me.Body('Th')
body_inertia1 = me.inertia(frame1, IUxx, IUyy, IUzz, IUxy, IUyz, IUzx)
body_inertia2 = me.inertia(frame2, IMxx, IMyy, IMzz, IMxy, IMyz, IMzx)
body_inertia3 = me.inertia(frame3, ILxx, ILyy, ILzz, ILxy, ILyz, ILzx)

clavicula = me.Body('Cl', mc1, m1, frame1, body_inertia1)
scapula = me.Body('Sc', mc2, m2, frame2, body_inertia2)
humerus = me.Body('Hu', mc3, m3, frame3, body_inertia3)

# creating joints
joint1 = me.SphericalJoint('J1', thorax, clavicula, 
                        child_point=rigid1C[0]*clavicula.x+rigid1C[1]*clavicula.y+rigid1C[2]*clavicula.z,
                        parent_point=rigid0P[0]*thorax.x+rigid0P[1]*thorax.y+rigid0P[2]*thorax.z,
                        coordinates = [q1, q2, q3],
                        speeds = [u1, u2, u3],rot_order='123', rot_type = 'body')
joint2 = me.SphericalJoint('J2', clavicula,  scapula,  
                        child_point=rigid2C[0]*scapula.x+rigid2C[1]*scapula.y+rigid2C[2]*scapula.z,
                        parent_point=rigid1P[0]*clavicula.x+rigid1P[1]*clavicula.y+rigid1P[2]*clavicula.z, 
                        coordinates = [q4, q5, q6],
                        speeds = [u4, u5, u6],rot_order='123', rot_type = 'body')
joint3 = me.SphericalJoint('J3', scapula,  humerus,  
                        child_point=rigid3C[0]*humerus.x+rigid3C[1]*humerus.y+rigid3C[2]*humerus.z,
                        parent_point=rigid2P[0]*scapula.x+rigid2P[1]*scapula.y+rigid2P[2]*scapula.z, 
                        coordinates = [q7, q8, q9],
                        speeds = [u7, u8, u9],rot_order='123', rot_type = 'body')

# applying gravitational forces in Y direction
clavicula.apply_force(-thorax.y*clavicula.mass*g)
scapula.apply_force(-thorax.y*scapula.mass*g)
humerus.apply_force(-thorax.y*humerus.mass*g)

# linear springs and dampings in joints
T1 = -k*q1*frame1.x-c*u1*frame1.x
T2 = -k*q2*frame1.y-c*u2*frame1.y
T3 = -k*q3*frame1.z-c*u3*frame1.z
T4 = -k*q4*frame2.x-c*u4*frame2.x
T5 = -k*q5*frame2.y-c*u5*frame2.y
T6 = -k*q6*frame2.z-c*u6*frame2.z
T7 = -k*q7*frame3.x-c*u7*frame3.x
T8 = -k*q8*frame3.y-c*u8*frame3.y
T9 = -k*q9*frame3.z-c*u9*frame3.z
clavicula.apply_torque(T1+T2+T3,thorax)
scapula.apply_torque(T4+T5+T6,clavicula)
humerus.apply_torque(T7+T8+T9,scapula)

In [3]:
# Contact between scapula and thorax

# contact points 
contact_point1 = me.Point('CP1')
contact_point1.set_pos(scapula.masscenter,x_cont1*scapula.x+y_cont1*scapula.y  +z_cont1*scapula.z)
contact_point1.set_vel(scapula.frame,0) # point is fixed in scapula

contact_point2 = me.Point('CP2')
contact_point2.set_pos(scapula.masscenter,x_cont2*scapula.x+y_cont2*scapula.y  +z_cont2*scapula.z)
contact_point2.set_vel(scapula.frame,0) # point is fixed in scapula

## contact forces

# Distances between contact points and thorax frame
x_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.x)+x_cont1*scapula.x.dot(thorax.x)+y_cont1*scapula.y.dot(thorax.x)+z_cont1*scapula.z.dot(thorax.x)
y_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.y)+x_cont1*scapula.x.dot(thorax.y)+y_cont1*scapula.y.dot(thorax.y)+z_cont1*scapula.z.dot(thorax.y)
z_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.z)+x_cont1*scapula.x.dot(thorax.z)+y_cont1*scapula.y.dot(thorax.z)+z_cont1*scapula.z.dot(thorax.z)
x_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.x)+x_cont2*scapula.x.dot(thorax.x)+y_cont2*scapula.y.dot(thorax.x)+z_cont2*scapula.z.dot(thorax.x)
y_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.y)+x_cont2*scapula.x.dot(thorax.y)+y_cont2*scapula.y.dot(thorax.y)+z_cont2*scapula.z.dot(thorax.y)
z_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.z)+x_cont2*scapula.x.dot(thorax.z)+y_cont2*scapula.y.dot(thorax.z)+z_cont2*scapula.z.dot(thorax.z)

# Contact forces
f1 = ((x_pos1-mx)/ax)**2+((y_pos1-my)/ay)**2+((z_pos1-mz)/az)**2-1
F1 = 1/2*(f1-sp.sqrt(f1**2+eps**2))
Fx1 = -k_contact*(x_pos1-mx)*(ax**2+ay**2+az**2)/(ax**2)*F1
Fy1 = -k_contact*(y_pos1-my)*(ax**2+ay**2+az**2)/(ay**2)*F1
Fz1 = -k_contact*(z_pos1-mz)*(ax**2+ay**2+az**2)/(az**2)*F1

f2 = ((x_pos2-mx)/ax)**2+((y_pos2-my)/ay)**2+((z_pos2-mz)/az)**2-1
F2 = 1/2*(f2-sp.sqrt(f2**2+eps**2))
Fx2 = -k_contact*(x_pos2-mx)*(ax**2+ay**2+az**2)/(ax**2)*F2
Fy2 = -k_contact*(y_pos2-my)*(ax**2+ay**2+az**2)/(ay**2)*F2
Fz2 = -k_contact*(z_pos2-mz)*(ax**2+ay**2+az**2)/(az**2)*F2

# applying contact forces to contact points in thorax frame
scapula.apply_force(thorax.x*Fx1+thorax.y*Fy1+thorax.z*Fz1,contact_point1)
scapula.apply_force(thorax.x*Fx2+thorax.y*Fy2+thorax.z*Fz2,contact_point2)


In [9]:
# method for solving EoMs
method = me.JointsMethod(thorax,joint1,joint2,joint3)
method.form_eoms()
mm = method.mass_matrix_full
fo = method.forcing_full

In [16]:
# substituting coords (x1..x9) and speeds (x10..x18)
x = sp.symbols('x1:19')
subs_q = {q1:x[0],q2:x[1],q3:x[2],q4:x[3],q5:x[4],q6:x[5],q7:x[6],q8:x[7],q9:x[8]}
subs_u = {u1:x[9],u2:x[10],u3:x[11],u4:x[12],u5:x[13],u6:x[14],u7:x[15],u8:x[16],u9:x[17]}
mm_subs = me.msubs(mm,subs_q,subs_u)
fo_subs = me.msubs(fo,subs_q,subs_u)

In [11]:
me.mprint(sp.octave_code(mm_subs,assign_to = 'mm'))

mm = [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 m1.*((-rigid1Cx.*sin(x2) + rigid1Cz.*cos(x2).*cos(x3)).^2 + (rigid1Cy.*sin(x2) + rigid1Cz.*sin(x3).*cos(x2)).^2 + (-rigid1Cx.*sin(x3).*cos(x2) - rigid1Cy.*cos(x2).*cos(x3)).^2) + m2.*((rigid2Cx.*((sin(x4).*cos(x6) + sin(x5).*sin(x6).*cos(x4)).*sin(x2) - (-sin(x4).*sin(x5).*sin(x6) + cos(x4).*cos(x6)).*sin(x3).*cos(x2) - sin(x6).*cos(x2).*cos(x3).*cos(x5)) - rigid2Cy.*((sin(x4).*sin(x6) - sin(x5).*cos(x4).*cos(x6)).*sin(x2) - (sin(x4).*sin(x5).*cos(x6) + sin(x6).*cos(x4)).*sin(x3).*cos(x2) + cos(x2).*cos(x3).*cos(x5).*cos(x6))).^2 + (rigid2Cx.*((sin(x4).*cos(x6) + sin(x5).*sin(x6).*cos(x4)).*sin(x2) - (-sin(x4).*sin(x5).*sin(x6) + cos(x

In [12]:
me.mprint(sp.octave_code(fo_subs,assign_to='fo'))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
## reaction forces in scapula

x1, x2, x3, dx1, dx2, dx3, ddx1, ddx2, ddx3= sp.symbols('x1 x2 x3 dx1 dx2 dx3 ddx1 ddx2 ddx3')
x4, x5, x6, dx4, dx5, dx6, ddx4, ddx5, ddx6= sp.symbols('x4 x5 x6 dx4 dx5 dx6 ddx4 ddx5 ddx6')
x7, x8, x9, dx7, dx8, dx9, ddx7, ddx8, ddx9= sp.symbols('x7 x8 x9 dx7 dx8 dx9 ddx7 ddx8 ddx9')

# reaction forces in scapula frame from accelerations
R_vec = (humerus.masscenter.acc(thorax.frame) + g*thorax.y) * humerus.mass

Rx = R_vec.dot(scapula.x)
Ry = R_vec.dot(scapula.y)
Rz = R_vec.dot(scapula.z)

# q1d = u1 ...
subsq1 = {q1:x1,q2:x2,q3:x3,sp.Derivative(q1, t):dx1,sp.Derivative(q2, t):dx2,sp.Derivative(q3, t):dx3,sp.Derivative(q1, (t, 2)):ddx1,sp.Derivative(q2, (t, 2)):ddx2,sp.Derivative(q3, (t, 2)):ddx3}
subsq2 = {q4:x4,q5:x5,q6:x6,sp.Derivative(q4, t):dx4,sp.Derivative(q5, t):dx5,sp.Derivative(q6, t):dx6,sp.Derivative(q4, (t, 2)):ddx4,sp.Derivative(q5, (t, 2)):ddx5,sp.Derivative(q6, (t, 2)):ddx6}
subsq3 = {q7:x7,q8:x8,q9:x9,sp.Derivative(q7, t):dx7,sp.Derivative(q8, t):dx8,sp.Derivative(q9, t):dx9,sp.Derivative(q7, (t, 2)):ddx7,sp.Derivative(q8, (t, 2)):ddx8,sp.Derivative(q9, (t, 2)):ddx9}
subsu1 = {u1:dx1,u2:dx2,u3:dx3,sp.Derivative(u1, t):ddx1,sp.Derivative(u2, t):ddx2,sp.Derivative(u3, t):ddx3}
subsu2 = {u4:dx4,u5:dx5,u6:dx6,sp.Derivative(u4, t):ddx4,sp.Derivative(u5, t):ddx5,sp.Derivative(u6, t):ddx6}
subsu3 = {u7:dx7,u8:dx8,u9:dx9,sp.Derivative(u7, t):ddx7,sp.Derivative(u8, t):ddx8,sp.Derivative(u9, t):ddx9}

Rx_subs = me.msubs(Rx,subsq1,subsq2,subsq3,subsu1,subsu2,subsu3)
Ry_subs = me.msubs(Ry,subsq1,subsq2,subsq3,subsu1,subsu2,subsu3)
Rz_subs = me.msubs(Rz,subsq1,subsq2,subsq3,subsu1,subsu2,subsu3)
R = sp.Matrix([Rx_subs, Ry_subs, Rz_subs])

me.mprint(sp.octave_code(R,assign_to = 'R'))

R = [g.*m3.*(-(sin(x4).*sin(x6) - sin(x5).*cos(x4).*cos(x6)).*sin(x1).*cos(x2) + (-sin(x1).*sin(x2).*sin(x3) + cos(x1).*cos(x3)).*(sin(x4).*sin(x5).*cos(x6) + sin(x6).*cos(x4)) + (sin(x1).*sin(x2).*cos(x3) + sin(x3).*cos(x1)).*cos(x5).*cos(x6)) + m3.*(sin(x4).*sin(x6) - sin(x5).*cos(x4).*cos(x6)).*(rigid1Cx.*(-ddx1.*sin(x3).*cos(x2) + ddx2.*cos(x3) + dx1.*dx2.*sin(x2).*sin(x3) - dx1.*dx3.*cos(x2).*cos(x3) - dx2.*dx3.*sin(x3)) - rigid1Cy.*(ddx1.*cos(x2).*cos(x3) + ddx2.*sin(x3) - dx1.*dx2.*sin(x2).*cos(x3) - dx1.*dx3.*sin(x3).*cos(x2) + dx2.*dx3.*cos(x3)) - rigid1Px.*(-ddx1.*sin(x3).*cos(x2) + ddx2.*cos(x3) + dx1.*dx2.*sin(x2).*sin(x3) - dx1.*dx3.*cos(x2).*cos(x3) - dx2.*dx3.*sin(x3)) + rigid1Py.*(ddx1.*cos(x2).*cos(x3) + ddx2.*sin(x3) - dx1.*dx2.*sin(x2).*cos(x3) - dx1.*dx3.*sin(x3).*cos(x2) + dx2.*dx3.*cos(x3)) - (-dx1.*sin(x3).*cos(x2) + dx2.*cos(x3)).*(rigid1Cy.*(dx1.*sin(x2) + dx3) - rigid1Cz.*(-dx1.*sin(x3).*cos(x2) + dx2.*cos(x3)) - rigid1Py.*(dx1.*sin(x2) + dx3) + rigid1Pz.*(-dx

In [7]:
clavicula.ang_vel_in(thorax.frame).dot(thorax.x)

(u1(t)*sin(q2(t)) + u3(t))*sin(q2(t)) - (-u1(t)*sin(q3(t))*cos(q2(t)) + u2(t)*cos(q3(t)))*sin(q3(t))*cos(q2(t)) + (u1(t)*cos(q2(t))*cos(q3(t)) + u2(t)*sin(q3(t)))*cos(q2(t))*cos(q3(t))